In [1]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert, convolve2d
from scipy.ndimage import gaussian_filter
from numpy.fft import rfftfreq, rfft, fft2, fftshift
from matplotlib import animation

np.random.seed(137)

H,W,STEPS = 128,128,300
SHIFT=1
GH_THETA=2
RANDOM_SPARKS=0.0
SMOOTH_SIGMA=0.8

def fibonacci_word(n_chars=4096):
    s="0"
    while len(s)<n_chars:
        s=s.replace("0","01").replace("1","0")
    return s[:n_chars]

def seed_from_bidgets(bidgets,H=128,W=128,shift=1):
    arr1d=np.array(list(bidgets[:W]),dtype=int)
    grid=np.zeros((H,W),dtype=int)
    for r in range(H):
        grid[r]=np.roll(arr1d,(r*shift)%W)
    return grid

def gh_step(state,theta=2):
    kernel=np.array([[1,1,1],[1,0,1],[1,1,1]])
    excited_neighbors=convolve2d((state==1).astype(int),kernel,mode='same',boundary='wrap')
    new_state=state.copy()
    new_state[(state==0)&(excited_neighbors>=theta)]=1
    new_state[state==1]=2
    new_state[state==2]=0
    return new_state

def run_gh(seed_binary,steps=300,theta=2,random_sparks=0.0):
    state=(seed_binary>0).astype(int)
    frames=[]
    for t in range(steps):
        if random_sparks>0 and t%20==0:
            mask=(np.random.rand(*state.shape)<random_sparks).astype(int)
            state[mask==1]=1
        frames.append(state.copy())
        state=gh_step(state,theta=theta)
    return np.array(frames)

def frames_to_phase(frames,sigma=0.8):
    T,H,W=frames.shape
    smoothed=np.zeros_like(frames,dtype=float)
    for t in range(T):
        smoothed[t]=gaussian_filter(frames[t].astype(float),sigma=sigma)
    analytic=hilbert(smoothed,axis=0)
    phase=np.angle(analytic)
    amp=np.abs(analytic)
    return smoothed,phase,amp

def dominant_wavelength(frame):
    F=np.abs(fftshift(fft2(frame)))
    cy,cx=np.array(F.shape)//2
    F[cy-3:cy+4,cx-3:cx+4]=0
    iy,ix=np.unravel_index(np.argmax(F),F.shape)
    ky=iy-cy; kx=ix-cx
    k=np.sqrt(kx**2+ky**2)+1e-9
    lam=min(frame.shape)/k
    return lam

def estimate_speed(frames):
    diffs=[np.mean(np.abs(frames[t+1]-frames[t])) for t in range(len(frames)-1)]
    return float(np.mean(diffs))

def dominant_period(frames):
    T=frames.shape[0]
    s=frames[:,frames.shape[1]//2,frames.shape[2]//2].astype(float)
    S=np.abs(rfft(s - s.mean()))**2
    f=rfftfreq(T,d=1.0)
    f1=f[1:][np.argmax(S[1:])]+1e-9
    return 1.0/f1

def R1(frames):
    period=dominant_period(frames)
    lam=dominant_wavelength(frames[len(frames)//2])
    v=estimate_speed(frames)
    return v/(lam/period), {"period":period,"lambda":lam,"v":v}

def make_animation(frames, path):
    fig,ax=plt.subplots(figsize=(5,5))
    im=ax.imshow(frames[0],cmap='gray',origin='lower',animated=True)
    ax.axis('off')
    def animate(i):
        im.set_data(frames[i]); return (im,)
    ani=animation.FuncAnimation(fig,animate,frames=len(frames),interval=50,blit=True)
    try:
        Writer=animation.writers['ffmpeg']
        writer=Writer(fps=20,metadata=dict(artist='ECHO'),bitrate=1200)
        ani.save(path,writer=writer)
        print("Saved",path)
    except Exception as e:
        print("FFMPEG not available; skipping MP4",e)
    plt.close(fig)

if __name__=="__main__":
    bidgets=fibonacci_word(max(H,W)*4)
    seed=seed_from_bidgets(bidgets,H,W,SHIFT)
    gh_frames=run_gh(seed,steps=STEPS,theta=GH_THETA,random_sparks=RANDOM_SPARKS)
    make_animation(gh_frames,"echo_excitable.mp4")
    r1, meta=R1(gh_frames)
    print("R1:",r1, meta)


Saved echo_excitable.mp4
R1: 0.0 {'period': np.float64(299.999910000027), 'lambda': np.float64(1.4142135623574699), 'v': 0.0}
